In [1]:
#Import Packages
import pandas as pd
import numpy as np
import os
import itertools
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime
import matplotlib
from herbie import Herbie
import pickle
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.dates as mdates
import pysolar.solar as solar
from geographiclib.geodesic import Geodesic
import xarray as xr
import pytz
import simplekml
import cartopy
import sys
sys.path.append('..')
import funcs.ac_funcs as ac
import funcs.plotting as plotting


pd.options.mode.chained_assignment = None


#Autoreload changed local modules
%load_ext autoreload
%autoreload 2

In [ ]:
#base_project_dir = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/'
base_project_dir = '/Users/agmeyer4/LAIR_1/'


#SLC SOURCES
slc_sources = dict(
    landfill = {'salt_lake_valley':{'lat':40.746,'lon':-112.042},
                 'trans_jordan':{'lat':40.55862,'lon':-112.053},
                 'davis':{'lat':41.114,'lon':-111.931},
                 'weber':{'lat':41.218,'lon':-111.99},
                 'bountiful':{'lat':40.911,'lon':-111.917}},
    ww_plant = {'central_valley':{'lat':40.7036613,'lon':-111.9141398},
                 'big_cottonwood':{'lat':40.6187424,'lon':-111.7824328},
                 'se_regional':{'lat':40.5411975,'lon':-111.8191652},
                 'south_valley':{'lat':40.5033357,'lon':-111.9187493},
                 'slc':{'lat':40.8030915,'lon':-111.9295899},
                 },
    refinery = {'Chevron':        {'lat':40.825,'lon':-111.924},
                  'Big West Oil':   {'lat':40.838,'lon':-111.920},
                  'Marathon':       {'lat':40.794,'lon':-111.909},
                  'Holly Refining': {'lat':40.887,'lon':-111.904},
                  'Silver Eagle':   {'lat':40.868,'lon':-111.910}}
    )

source_df = pd.DataFrame()
for sourcetype in slc_sources.keys():
    df = pd.DataFrame(slc_sources[sourcetype]).transpose()
    df['type'] = sourcetype
    source_df = pd.concat([source_df,df])

In [ ]:
class stilt_output_handler:
    def __init__(self,stilt_base_output_path):
        self.stilt_base_output_path = stilt_base_output_path
        self.byid_path = os.path.join(stilt_base_output_path,'by-id')
        self.id_list = os.listdir(self.byid_path)
        self.id_dicts = {key: {} for key in self.id_list}
    
    def get_id_path(self,id):
        id_path = os.path.join(self.byid_path,id)
        return id_path
    
    def id_has_nc(self,id):
        files = os.listdir(self.get_id_path(id))
        if any('foot.nc' in file for file in files): #check if the footprint is in the list of files by looking for substring
            self.id_dicts[id]['has_nc'] = True
            self.id_dicts[id]['nc_fname']= [i for i in files if 'foot.nc' in i][0] #TODO find a way to do this without the list? shouldn't be multple nc file but sloppy
        else:
            self.id_dicts[id]['has_nc'] = False
        return True
    
    def check_all_for_nc(self):
        for id in self.id_list:
            self.id_has_nc(id)
    
    def get_meta_from_fname(self,fname):
        dt_str,lon_str,lat_str,zagl_str,run_id = fname.split('_')[:-1]
        lon = float(lon_str)
        lat = float(lat_str)
        zagl = float(zagl_str)
        run_id = int(run_id)
        return {'dt_str':dt_str,'lon':lon,'lat':lat,'zagl':zagl,'run_id':run_id}

    def get_all_meta(self):
        for id in self.id_dicts.keys():
            try:
                fname = self.id_dicts[id]['nc_fname']
            except:
                continue
                
            self.id_dicts[id].update(self.get_meta_from_fname(fname))
            self.id_dicts[id]['full_nc_path'] = os.path.join(self.get_id_path(id),fname)
            self.id_dicts[id]['id']=id

In [ ]:
my_stilt_handler = stilt_output_handler(f'/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/STILT/20230711_stilt/out')
my_stilt_handler.check_all_for_nc()
my_stilt_handler.get_all_meta()

stilt_footprints_df = pd.DataFrame(my_stilt_handler.id_dicts).transpose()
stilt_footprints_df['dt'] = pd.to_datetime(stilt_footprints_df['dt_str']).dt.tz_localize('UTC')
stilt_footprints_df = stilt_footprints_df.set_index('dt')
stilt_footprints_df_good =stilt_footprints_df.dropna()
stilt_footprints_df_good.index = stilt_footprints_df_good.index.tz_convert('US/Mountain')
stilt_footprints_df_good = stilt_footprints_df_good.sort_values('run_id')

# em27_data_folder = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/SLC_EM27_ha_2022_2023_oof_v2_nasrin_correct'
# tz = 'US/Mountain'
# dt_range = {'dt1':'2023-07-11 08:00:00','dt2':'2023-07-11 20:00:00'}
# oof_manage = ac.oof_manager(em27_data_folder,tz)
# em27_df = oof_manage.load_oof_df_inrange(dt_range['dt1'],dt_range['dt2'],True)
# em27_df = em27_df.resample('T').mean()
# species = 'xch4(ppm)'

In [ ]:
stilt_footprints_df_good

In [ ]:
my_stilt_handler = stilt_output_handler(f'/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/STILT/20230809_stilt/out')
my_stilt_handler.check_all_for_nc()
my_stilt_handler.get_all_meta()

stilt_footprints_df = pd.DataFrame(my_stilt_handler.id_dicts).transpose()
stilt_footprints_df['dt'] = pd.to_datetime(stilt_footprints_df['dt_str']).dt.tz_localize('UTC')
stilt_footprints_df = stilt_footprints_df.set_index('dt')
stilt_footprints_df_good =stilt_footprints_df.dropna()
stilt_footprints_df_good.index = stilt_footprints_df_good.index.tz_convert('US/Mountain')
stilt_footprints_df_good = stilt_footprints_df_good.sort_values('run_id')

em27_data_folder = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/SLC_EM27_ha_2022_2023_oof_v2_nasrin_correct'
tz = 'US/Mountain'
dt_range = {'dt1':'2023-08-09 08:00:00','dt2':'2023-08-09 20:00:00'}
oof_manage = ac.oof_manager(em27_data_folder,tz)
em27_df = oof_manage.load_oof_df_inrange(dt_range['dt1'],dt_range['dt2'],True)
em27_df = em27_df.resample('T').mean()
species = 'xch4(ppm)'

In [ ]:
save_path = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Figures/oct2023/STILT/20230711/'

min_foot = 0.001
j = 0
for dt in stilt_footprints_df_good.index.unique():
    if j==0:
        j+=1
        continue
    sub_df = stilt_footprints_df_good.loc[stilt_footprints_df_good.index == dt]

    fp = sub_df.iloc[0]
    full_nc_path = fp['full_nc_path']
    ds = xr.open_dataset(full_nc_path)
    summed_da = ds.sum(dim='time')
    foot_sum = summed_da.foot

    for i in range(1,len(sub_df)):
        fp = sub_df.iloc[i]
        full_nc_path = fp['full_nc_path']
        ds = xr.open_dataset(full_nc_path)
        summed_da = ds.sum(dim='time')
        foot_da = summed_da.foot
        foot_sum = foot_sum + foot_da

    foot_sum_masked = foot_sum.where(foot_sum>min_foot)

    inst_lat = 40.766
    inst_lon = -111.847
    extent=[-112.3,-111.5,40.5,41.0]
    labsize = 14

    zoom = 0.2
    proj = ccrs.PlateCarree()
    fig = plt.figure(figsize=(10,10))
    ax = plt.axes(projection = proj)
    ax.set_extent(extent,crs=proj)

    request = cimgt.GoogleTiles(style='satellite')
    scale = np.ceil(-np.sqrt(2)*np.log(np.divide(zoom,350.0))) # empirical solve for scale based on zoom
    ax.add_image(request,int(scale))
    footplot = foot_sum_masked.plot(ax=ax,transform=proj,add_colorbar=False,alpha = 0.65)
    cbar = plt.colorbar(footplot,fraction=0.028)
    cbar.ax.tick_params(labelsize=labsize)
    cbar.set_label(label = 'STILT Footprint (not weighted)',size=labsize)

    colors = ['#CC3311','#0077BB','#F4BB44']
    i= 0
    for source_type in source_df.groupby('type').sum().index:
        minidf = source_df.loc[source_df['type'] ==source_type]
        ax.scatter(minidf['lon'],minidf['lat'],label=source_type,c=colors[i],s=40)
        i+=1

    ax.scatter(inst_lon,inst_lat,label='EM27',marker='x',c='k',s=70)
    t = ax.text(extent[1]-.17,extent[2]+.02,f'{dt.hour:02}:{dt.minute:02}{"MDT"}',fontsize=20,color='k')
    t.set_bbox(dict(facecolor='white', alpha=0.7))

    axins = inset_axes(ax,width='40%',height='20%',loc='lower left')
    axins.scatter(em27_df.index,em27_df[species],color = 'blue',zorder=3,s=1)
    # axins.errorbar(em27_df.index,em27_df[species],
    #                 yerr=em27_df[f'{species}_error'],ls='none',
    #                 ecolor='grey',alpha=0.2)
    axins.tick_params(labelsize = labsize)
    axins.set_ylabel(species,size = labsize)
    axins.vlines(dt,axins.get_ylim()[0],axins.get_ylim()[1],zorder = 10,linewidth=5,color = 'grey',alpha = 0.5)
    axins.xaxis.set_major_locator(mdates.HourLocator(interval=2))   
    axins.xaxis.set_major_formatter(mdates.DateFormatter('%H', tz = em27_df.index.tz))
    axins.set_xlabel(em27_df.index[0].strftime('%Z %b %d, %Y'),size = labsize)
    ax.legend(fontsize=labsize,loc = 0)
    plt.gcf().autofmt_xdate()
    plt.show()
    savename = f"{dt.hour:02}.png"
    fig.savefig(os.path.join(save_path,savename),dpi=500)
    


In [ ]:
min_foot = 0.00

for idx in range(len(stilt_footprints_df_good)):
#idx = 72
    fp = stilt_footprints_df_good.iloc[idx]
    full_nc_path = fp['full_nc_path']
    lat = fp['lat']
    lon = fp['lon']
    zagl = fp['zagl']
    dt_str = fp['dt_str']
    dt = fp.name
    id = fp.id

    inst_lat = 40.766
    inst_lon = -111.847

    ds = xr.open_dataset(full_nc_path)
    summed_da = ds.sum(dim='time')
    foot = summed_da.foot
    masked_foot = foot.where(foot>min_foot)

    extent=[-112.3,-111.5,40.5,41.0]


    species = 'xch4(ppm)'
    labsize = 14

    stilt_footprint_png_path = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Figures/'
    stilt_footprint_png_fname = f'{id}.png'

    print(id)
    zoom = 0.2
    proj = ccrs.PlateCarree()
    fig = plt.figure(figsize=(10,10))
    ax = plt.axes(projection = proj)
    ax.set_extent(extent,crs=proj)

    request = cimgt.GoogleTiles(style='satellite')
    scale = np.ceil(-np.sqrt(2)*np.log(np.divide(zoom,350.0))) # empirical solve for scale based on zoom
    ax.add_image(request,int(scale))
    #contour = ax.contourf(summed_da['lon'],summed_da['lat'],summed_da.foot,transform = proj , alpha = 0.9,vmin=min_foot)
    foot = masked_foot.plot(ax=ax,transform=proj,add_colorbar=False,alpha = 0.65)
    cbar = plt.colorbar(foot,fraction=0.033)
    cbar.ax.tick_params(labelsize=labsize)
    cbar.set_label(label = 'STILT Footprint',size=labsize)

    colors = ['#CC3311','#0077BB','#F4BB44']
    i= 0
    for source_type in source_df.groupby('type').sum().index:
        minidf = source_df.loc[source_df['type'] ==source_type]
        ax.scatter(minidf['lon'],minidf['lat'],label=source_type,c=colors[i],s=40)
        i+=1

    t = ax.text(extent[1]-.2,extent[2]+.02,f'{dt.month}-{dt.day} {dt.hour}:{dt.minute:02}{"MDT"}\n{zagl}m agl',fontsize=20,color='k')
    t.set_bbox(dict(facecolor='white', alpha=0.7))

    ax.scatter(inst_lon,inst_lat,label='EM27',marker='x',c='k',s=70)

    # axins = inset_axes(ax,width='40%',height='20%',loc='lower left')
    # axins.scatter(em27_df.index,em27_df[species],color = 'grey',zorder=3,label='True Data Rate',s=1)
    # axins.errorbar(em27_df.index,em27_df[species],
    #                 yerr=em27_df[f'{species}_error'],ls='none',
    #                 ecolor='grey',alpha=0.2)
    # axins.tick_params(labelsize = labsize)
    # axins.set_ylabel(species,size = labsize)
    # axins.vlines(dt,axins.get_ylim()[0],axins.get_ylim()[1],zorder = 10,linewidth=5)
    # axins.xaxis.set_major_formatter(mdates.DateFormatter('%H', tz = em27_df.index.tz))
    # axins.set_xlabel(em27_df.index[0].strftime('%Z %b %d, %Y'),size = labsize)
    # plt.gcf().autofmt_xdate()

    ax.legend(fontsize=labsize)
    plt.show()
    #fig.savefig(os.path.join(stilt_footprint_png_path,stilt_footprint_png_fname),bbox_inches='tight')

# Stilt loader with id scheme

In [ ]:
all_stilt_path = '/uufs/chpc.utah.edu/common/home/lin-group15/agm/STILT_runs/'
date_run_dir = '20230711_stilt'
